<a href="https://colab.research.google.com/github/ucfilho/raianars_adjust_RTC/blob/master/RTC_adjust_v_05_mar_08_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
from scipy.optimize import differential_evolution
import numpy as np

In [0]:
# Funcoes utilizadas neste codigo
# Area(U,tempAtual,tempInicial,...): constroi matriz de areas trocadores 
#                                    entra duas correntes sendo chamada no loop
# buildMatriz(x): constroi a matriz deltaT e binaria (logica de trocas)
#
# Temperatura(matrizDeltaT,...): constroi as temperaturas e cargas termicas

In [0]:
def buildMatriz(x):
  global Trocas,D, tempInicial, tempAtual 
  cols=len(tempInicial) 
  Ind_I=np.zeros((D,Trocas)).astype(np.int64)
  Ind_J=np.zeros((D,Trocas)).astype(np.int64)
  Tin=np.zeros((D,Trocas))
  Tout=np.zeros((D,Trocas))
  deltaT=np.zeros((D,Trocas))
  for i in range(D):
    for j in range(Trocas):
      Ind_I[i,j]=round(x[i,j])
      Ind_J[i,j]=round(x[i,j+Trocas])
      deltaT[i,j]=x[i,j+2*Trocas] 
    for jj in range(cols):
      Tin[i,jj]=tempInicial[jj]
      Tout[i,jj]=tempAtual[jj]
  return Ind_I,Ind_J,deltaT, Tin, Tout

In [0]:
#def Temperatura_Carga(mCp,matrizDeltaT,matrizBinaria,tempInicial):
def Temperatura_Carga(mCp,Ind_I,Ind_J,deltaT,tempInicial,tempAtual):
  global Trocas,D
  # cols=len(mCp)
  # i represent hot stream  and j represent cold stream 
  # k represent sequential heat exchanges
  cargaTermica=np.zeros((Trocas))
  contador=0
  cont_uso=0
  erro=np.zeros((D)) # valor erro se torna erro=1 quando ha algum erro e vai penalizar
  for ki in range(D):
    for kj in range(Trocas):
      k=kj
      i=Ind_I[ki,kj]
      j=Ind_J[ki,kj]
      print('=================')
      print(i,j)
      print('mCp[i]',mCp[i])
      print(deltaT[ki,kj])
      cargaTermica[k]=mCp[i]*deltaT[ki,kj]
      tempSaidaQ=tempInicial[ki,i]+deltaT[ki,kj]
      tempSaidaF=-cargaTermica[k]/(mCp[j])+tempAtual[ki,j]
      flag=0 # flag : indica que nao entra no teste logico
      # usei flag no lugar q ifs pq a identacao ficou melhor assim
      if(Tin[ki,i] > Tin[ki,j]): #Tquente maior Tfria: vaores atuais
        if(tempSaidaQ > tempSaidaF): #Tquente maior Tfria: saidas
          if(contador < cols):
            flag=1
            
      if(flag>0):
        tempAtual[i]=tempSaidaQ
        tempAtual[j]=tempSaidaF
      else:
        erro[ki]=1


  return tempAtual,cargaTermica,erro

In [0]:
def Area(U,tempAtual,tempInicial,Ind_I,Ind_J,deltaT):
  global Trocas,mCp,D
  
  Area=[]
  for ki in range(D):
    for kj in range(Trocas):
      k=ki
      i=int(Ind_I[ki,kj])
      j=int(Ind_J[ki,kj])
      Carga=mCp[k]*deltaT[k]
      tempSaidaF=tempAtual[j]
      tempSaidaQ=tempAtual[i]
      deltaT1=tempInicial[i]-tempSaidaF
      deltaT2=tempSaidaQ-tempInicial[j]
      try:
        A=(deltaT1-deltaT2)
        B=np.log((deltaT1/deltaT2))
        deltaTMediaLog=A/B
      except:
        deltaTMediaLog=1e99
  
      if(deltaTMediaLog==1e99):
        AreaCalc=1e99
      else:
        AreaCalc=abs(Carga/(U*deltaTMediaLog))

      if(tempAtual[i]==tempInicial[i]):
            AreaCalc=0

      if(tempAtual[j]==tempInicial[j]):
        AreaCalc=0
            
      Area.append(AreaCalc)

  return Area

In [0]:
def Utilidades(tempAlvo,tempAtual,mCp):

  cols=len(mCp)
  cargaUQ=0
  cargaUF=0
  
  '''
  caso I cita TUQin=553 e TQout=552, TUFin=303 TUFout=353
  '''
  
  TUQin=553
  TUQout=552
  TUFin=303 
  TUFout=353
  areaTrocador=[]

  for j in range(cols): 
    delta=tempAlvo[j]-tempAtual[j]
    if(delta>0):#usa UQ
      cargaUQ=cargaUQ+mCp[j]*delta
      deltaT1=(TUQin-tempAlvo[j])
      deltaT2=(TUQout-tempAtual[j])

      try:
        A=deltaT1-deltaT2
        B=np.log(deltaT1/deltaT2)
        deltaTMediaLog=A/B
      except:
        deltaTMediaLog=1e99

      if(deltaTMediaLog==1e99):
        areaTrocador.append(1e99)
      else:
        areaTrocador.append((abs(mCp[j]*(delta)/(U*deltaTMediaLog))))

    elif(delta==0):#nao usa UF e UQ
      deltaTMediaLog=0
      areaTrocador.append(0) 

    else:#usa UF
      deltaT1=(tempAtual[j]-TUFout)
      deltaT2=(tempAlvo[j]-TUFin)     
      cargaUF=cargaUF+mCp[j]*(-delta)

      #UF -> água resfriada -> entra a 303K e sai a 353K
      try:
        A=deltaT1-deltaT2
        B=np.log(deltaT1/deltaT2)
        deltaTMediaLog=A/B
      except:
        deltaTMediaLog=1e99
      
      if(deltaTMediaLog==1e99):
        areaTrocador.append(1e99)
      else:
        areaTrocador.append((abs(mCp[j]*(-delta)/(U*deltaTMediaLog))))

  return areaTrocador, cargaUF,cargaUQ 
  # areaTrocador contem a soma das areas
  # Trocadores a area associada a cada corrente
  # cargaUF e cargaUQ kW gasto

In [0]:
def CustoProcesso(areaTrocadores,areaUtilidades,cargaUF,cargaUQ):
  global coefCustoUQ,coefCustoUF
  costTrocadores=0
  for it in areaTrocadores:
    for wii in areaUtilidades:
      if(it<1e99):
        if(wii<1e99):
          costTrocadores=costTrocadores+300*(it**0.5+wii**0.5)
        else:
          costTrocadores=1e99
          break
        
  costUQ=coefCustoUQ*cargaUQ
  costUF=coefCustoUF*cargaUF

  if(costTrocadores==1e99):
    cost=1e99
  else:
    cost=costUQ+costUF+costTrocadores
  return cost

In [0]:
#FUN calcula o TAC para um determinado arranjo
#  cada calculo é feito em uma funcao a parte Fun usa todas funcoes 

def FUN(x):

  # global mCp,tempAtual,tempAlvo,U,coefCustoUQ,coefCustoUF
  global mCp,tempAtual,tempAlvo,tempInicial,U,coefCustoUQ,coefCustoUF
  global D
  
  funr=np.zeros((D))


  cols=len(mCp)
  Ind_I,Ind_J,deltaT, Tin, Tout=buildMatriz(x) # transforma x em matriz logica e troca
  tempAtual,Carga,Erro=Temperatura_Carga(mCp,Ind_I,Ind_J,deltaT,Tin)
  areaTrocadores =Area(U,tempAtual,tempInicial,Ind_I,Ind_J,deltaT)
  areaUtilidade, cargaUF,cargaUQ = Utilidades(tempAlvo,tempAtual,mCp)
  cost=CustoProcesso(areaTrocadores,areaUtilidade,cargaUF,cargaUQ)
  funr=cost

  for k in range(D):
    if(Erro[k]>0):
      funr[k]=1e99

  funr = np.where(np.isnan(cost), 1e99, cost)

  return funr

In [0]:
#********************PROGRAMA PRINCIPAL*************************

NPAR=50 #Número de partículas
ITE=5000 #Número de iterações
PAR=12 #Número de parâmetros a ser otimizados

MAX=[0,0,0,0,0,0,1,1,1,1,1,1]
MIN=[-300,-300,-300,-300,-300,-300,0,0,0,0,0,0]
mCp=[4,2,1.5,3] #[kW/K]
tempInicial=[453,393,523,533] #[K]
#tempAtual=[453,393,523,533] #[K]
tempAlvo=[513,508,403,433] #[K]
U=0.2 #[kW/(m2.K)]

coefCustoUQ=110 #[$/(kW.ano)]
coefCustoUF=12.2 #[$/(kW.ano)]

bounds = []
for i in range(12):
  bounds.append((MAX[i],MIN[i]))



In [10]:
#MAX=[0,0,0,0,0,0,1,1,1,1,1,1]
#MIN=[-300,-300,-300,-300,-300,-300,0.3,0.3,0.3,0.3,0.3,0.3]
Trocas=4

D=5

cols=Trocas*3 # tres linhas i,j,deltaT
MAX=np.zeros((cols))
MIN=np.zeros((cols))

for k in range(cols):
  if(k>(Trocas*2-1)):
    MIN[k]=-100
  else:
    MAX[k]=(Trocas-1)

x=np.zeros((D,cols))

for i in range(D):
  for j in range(cols):
    r=np.random.random()
    x[i,j]=r*(MAX[j]-MIN[j])+MIN[j]

Ind_I=np.zeros((Trocas))
Ind_J=np.zeros((Trocas))
deltaT=np.zeros((Trocas))
for i in range(D):
  for j in range(Trocas):
    Ind_I[j]=round(x[i,j])
    Ind_J[j]=round(x[i,j+Trocas])
    deltaT[j]=x[i,j+2*Trocas]

# print(x) ok



[[  0.89721648   0.56453701   0.19000916   1.32518427   1.00324057
    2.21312628   2.37650187   0.65146936 -85.49232884 -25.72516723
  -78.79152406 -85.48939154]
 [  0.11972106   2.75971467   1.70436392   2.16100547   1.17114579
    2.63664105   0.48104885   0.12810373 -86.78406055 -89.77410223
  -95.90457962 -57.07149572]
 [  0.55384143   1.40950122   0.85654915   2.76282368   1.80266395
    1.67302342   2.07265482   1.4298793  -71.98871489 -78.84452364
  -30.16499802 -91.47767092]
 [  1.37861466   1.08781388   0.84755077   0.35535041   2.22151793
    1.44779158   1.58461844   2.13532964 -91.50901235 -43.38463964
  -49.93476939 -97.47928134]
 [  1.76919204   2.36868922   0.55393427   1.74288428   0.35185621
    0.87662948   0.90160745   0.31734605 -12.68558102 -95.16017733
  -32.9135168   -2.53255088]]


In [11]:
Ind_I,Ind_J,deltaT=buildMatriz(x) # transforma x em matriz logica e troca
print(Ind_I)
print(Ind_J)
print(deltaT)

[[1 1 0 1]
 [0 3 2 2]
 [1 1 1 3]
 [1 1 1 0]
 [2 2 1 2]]
[[1 2 2 1]
 [1 3 0 0]
 [2 2 2 1]
 [2 1 2 2]
 [0 1 1 0]]
[[-85.49232884 -25.72516723 -78.79152406 -85.48939154]
 [-86.78406055 -89.77410223 -95.90457962 -57.07149572]
 [-71.98871489 -78.84452364 -30.16499802 -91.47767092]
 [-91.50901235 -43.38463964 -49.93476939 -97.47928134]
 [-12.68558102 -95.16017733 -32.9135168   -2.53255088]]


In [12]:

Custo=FUN(x)
print(Custo)


1 1
mCp[i] 2
-85.49232883708355


IndexError: ignored

In [0]:
print(FUN(x))

In [0]:
print(x)

In [0]:
cols=len(mCp)
UsoFuncao=[]
x=[-99.214722,59.99994,146.225,4.2108538,15.99999,0, 0.9500049,1, 1,0.9103451, 1,0.4786627]
matrizDeltaT,matrizBinaria=buildMatriz(x) # transforma x em matriz logica e troca
tempAtual,Carga,Erro=Temperatura_Carga(mCp,matrizDeltaT,matrizBinaria,tempInicial)
areaTrocadores =Area(U,tempAtual,tempInicial,Carga,matrizDeltaT)
areaUtilidade, cargaUF,cargaUQ = Utilidades(tempAlvo,tempAtual,mCp)
print('estou aqui');print(areaUtilidade);print(areaTrocadores)
custoTotal=CustoProcesso(areaTrocadores,areaUtilidade,cargaUF,cargaUQ)
print('Custo total');print(custoTotal);
cost=CustoProcesso(areaTrocadores,areaUtilidade,cargaUF,cargaUQ)
print('Custo processo');print(cost);
print('cont_uso');print(UsoFuncao)

In [0]:
UsoFuncao=[]
fchoice=FUN
result = differential_evolution(fchoice, bounds, maxiter=ITE, popsize=NPAR)
print('temperatura atual=',tempAtual)
print(result.x, result.fun)
print(tempAtual)
GBEST=result.x
BEST=result.fun
matrizTrocaBest,matrizBinaria=MATBEST(GBEST,mCp,tempAtual)
matrizGrafico=MATGRAF(matrizTrocaBest,mCp,tempAtual)

print('Melhor solução para %d iterações com %d partículas \n'%(ITE,NPAR));print(GBEST)
print('Valor da função f(gbest)=%f \n'%BEST)
print('matriz grafico');print(matrizGrafico)
print('cont_uso');print(UsoFuncao)

In [0]:
cols=len(mCp)
matrizDeltaT,matrizBinaria=buildMatriz(GBEST,cols) # transforma x em matriz logica e troca
tempAtual,cargaTermica,erro=Temperatura_Carga(mCp,matrizDeltaT,matrizBinaria,tempInicial)
areaTrocadores =Area(U,tempAtual,tempInicial,cargaTermica,matrizDeltaT)

print('temperatura inicial');print(tempInicial)
print('temperatura alvo');print(tempAlvo)
print('temperatura atual');print(tempAtual)
print('Area trocadores');print(areaTrocadores)

In [0]:
print(cols)
print(GBEST)

In [0]:
matrizDeltaT,matrizBinaria=buildMatriz(GBEST,cols) # transforma x em matriz logica e troca
print('Matriz de Trocas Térmicas \n')
print(matrizDeltaT)
print('Matriz Binária \n')
print(matrizBinaria)